In [1]:
from grafos import Accion
from grafos import Estado
from grafos import Nodo
from grafos import Problema

Crear métodos auxiliares

In [2]:
def crear_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    raiz.costo = 0
    raiz.heuristicas = problema.heuristicas[estado_raiz.nombre]
    raiz.valores = dict(raiz.heuristicas.items())
    return raiz

def crear_nodo_hijo(problema, padre, accion, agregar = True):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    costo = padre.costo
    costo += problema.costo_accion(padre.estado, accion)
    hijo.costo = costo
    hijo.heuristicas = problema.heuristicas[hijo.estado.nombre]
    hijo.valores = {estado: heuristica + hijo.costo for estado, heuristica in hijo.heuristicas.items()}
    if agregar:
        hijo.padre = padre
        padre.hijos.append(hijo)
    return hijo

def sacar_siguiente(frontera, metrica = 'valor', criterio = 'menor', objetivos = None):
    mejor = frontera[0]
    for nodo in frontera[1:]:
        for objetivo in objetivos:
            if metrica == 'valor':
                valor_nodo = nodo.valores[objetivo.nombre]
                valor_mejor = mejor.valores[objetivo.nombre]
                if (criterio == 'menor' and valor_nodo < valor_mejor):
                    mejor = nodo
                elif (criterio == 'mayor' and valor_nodo > valor_mejor):
                    mejor = nodo
            elif metrica == 'heuristica':
                heuristica_nodo = nodo.heuristicas[objetivo.nombre]
                heuristica_mejor = mejor.heuristicas[objetivo.nombre]
                if (criterio == 'menor' and heuristica_nodo < heuristica_mejor):
                    mejor = nodo
                elif (criterio == 'mayor' and heuristica_nodo > heuristica_mejor):
                    mejor = nodo
            if metrica == 'costo':
                if (criterio == 'menor' and nodo.costo_camino < mejor.costo_camino):
                    mejor = nodo
                elif (criterio == 'mayor' and nodo.costo_camino > mejor.costo_camino):
                    mejor = nodo
    frontera.remove(mejor)
    return mejor         


def muestra_solucion(objetivo = None, problema_resolver = None):
    if not objetivo:
        print("No hay solución")
        return
    nodo = objetivo
    while nodo:
        msg = "Estado {0}, Valor {1}"
        estado = nodo.estado.nombre
        valores = [nodo.valores[objetivo.nombre]
                   for objetivo
                   in problema_resolver.estado_objetivo]
        valor = min(valores)
        print(msg.format(estado, valor))
        msg = "  Costo: {0}"
        costo_total = nodo.costo
        print(msg.format(costo_total))
        msg = "  Heurística: {0}"
        heuristicas_objetivos = [nodo.heuristicas[objetivo.nombre]
                                 for objetivo
                                 in problema_resolver.estado_objetivo]
        heuristica = min(heuristicas_objetivos)
        print(msg.format(heuristica))
        if nodo.accion:
            accion = nodo.accion.nombre
            padre = nodo.padre.estado
            costo = problema_resolver.costo_accion(padre, nodo.accion)
            if accion:
                msg = "<--- {0} [{1}] ---"
                print(msg.format(accion, costo))
        nodo = nodo.padre

Crear la función que ejecuta el problema

In [3]:
def voraz(problema):
    raiz = crear_nodo_raiz(problema)
    frontera = [raiz, ]
    explorados = set()
    while True:
        if not frontera:
            return None
        nodo = sacar_siguiente(frontera, 'heuristica',
                               objetivos=problema.estado_objetivo, criterio= 'mayor')
        if problema.es_objetivo(nodo.estado):
            return nodo
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crear_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if hijo.estado in explorados or hijo.estado in estados_frontera:
                buscar = [nodo for nodo in frontera
                          if nodo.estado == hijo.estado]
                if buscar:
                    heuristic_hijo = [hijo.heuristicas[objetivo.nombre]
                                      for objetivo
                                      in problema.estado_objetivo]
                    heuristic_buscar = [buscar[0].heuristicas[objetivo.nombre]
                                        for objetivo
                                        in problema.estado_objetivo]
                    minimo_hijo = min(heuristic_hijo)
                    minimo_buscar = min(heuristic_buscar)
                    if minimo_hijo < minimo_buscar:
                        indice = frontera.index(buscar[0])
                        frontera[indice] = hijo
            else:
                frontera.append(hijo)

# Crear la estructura del problema

In [4]:
# Lo mismo que en el de amplitud, pero en lugar de una cola, usamos una pila
if __name__ == '__main__':
    # vamos a crear las acciones que puede hacer el agente
    accN = Accion('N')
    accS = Accion('S')
    accE = Accion('E')
    accO = Accion('O')
    accNE = Accion('NE')
    accSE = Accion('SE')
    accSO = Accion('SO')
    accNO = Accion('NO')

    # Vamos a crear los estados
    cedi = Estado('cedi', [accSE, accSO])
    tienda_1 = Estado('tienda_1', [accS, accSE])
    tienda_2 = Estado('tienda_2', [accO, accS, accSE])
    tienda_3 = Estado('tienda_3', [accE])
    tienda_4 = Estado('tienda_4', [accE])
    tienda_5 = Estado('tienda_5', [accNE])
    parqueadero = Estado('parqueadero', [])

    # Acciones que puede hacer
    acciones = {'cedi':{'SO':tienda_1, 'SE':tienda_2},
                'tienda_1':{'S':tienda_3, 'SE':tienda_4},
                'tienda_2':{'O':tienda_1, 'S':tienda_4, 'SE':parqueadero},
                'tienda_3':{'E':tienda_4},
                'tienda_4':{'E':tienda_5},
                'tienda_5':{'NE':parqueadero}}
    
    costos = {'cedi':{'SO':15, 'SE':50},
                'tienda_1':{'S':20, 'SE':16},
                'tienda_2':{'O':18, 'S':25, 'SE':42},
                'tienda_3':{'E':8},
                'tienda_4':{'E':12},
                'tienda_5':{'NE':30}
                }
    
    heuristicas = {'cedi':{'cedi':0, 'tienda_1':85, 'tienda_2':87, 'tienda_3':80, 'tienda_4':85, 'tienda_5':88, 'parqueadero':99},
                   'tienda_1':{'cedi':85, 'tienda_1':0, 'tienda_2':90, 'tienda_3':70, 'tienda_4':80, 'tienda_5':90, 'parqueadero':90},
                   'tienda_2':{'cedi':87, 'tienda_1':90, 'tienda_2':0, 'tienda_3':80, 'tienda_4':83, 'tienda_5':90, 'parqueadero':95},
                   'tienda_3':{'cedi':80, 'tienda_1':70, 'tienda_2':80, 'tienda_3':0, 'tienda_4':90, 'tienda_5':83, 'parqueadero':95},
                   'tienda_4':{'cedi':85, 'tienda_1':80, 'tienda_2':83, 'tienda_3':90, 'tienda_4':0, 'tienda_5':85, 'parqueadero':100},
                   'tienda_5':{'cedi':88, 'tienda_1':90, 'tienda_2':90, 'tienda_3':83, 'tienda_4':85, 'tienda_5':0, 'parqueadero':100},
                   'parqueadero':{'cedi':99, 'tienda_1':90, 'tienda_2':95, 'tienda_3':95, 'tienda_4':100, 'tienda_5':100, 'parqueadero':0}
                   }

In [5]:
objetivo = [parqueadero]
problema = Problema(cedi, objetivo, acciones, costos, heuristicas)
solucion = voraz(problema)
muestra_solucion(solucion, problema)

Estado parqueadero, Valor 92
  Costo: 92
  Heurística: 0
<--- SE [42] ---
Estado tienda_2, Valor 145
  Costo: 50
  Heurística: 95
<--- SE [50] ---
Estado cedi, Valor 99
  Costo: 0
  Heurística: 99
